# Pre-Requisites

In [4]:
!pip3 install -U sentence-transformers

     |████████████████████████████████| 85 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.5 MB 42.9 MB/s eta 0:00:01
     |████████████████████████████████| 31.2 MB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 43.4 MB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 75.6 MB/s eta 0:00:01
     |████████████████████████████████| 19.1 MB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 306 kB 78.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 50.8 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /home/ubuntu/gitrepos/semantic-search/ENV/bin/python3.8 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-wg2kq9lv/sentence-transformers/setup.py'"'"'; __file__='"'"'/tmp/pip-install-wg2kq9lv/sentence-transformers/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.rea

In [7]:
!pip3 install pinecone-client

     |████████████████████████████████| 175 kB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 269 kB 62.0 MB/s eta 0:00:01


## Query Generation

In [9]:
import pandas as pd

query_passage_df = pd.read_csv('query_passage.csv')
query_passage_df.head()

,Unnamed: 0,query,passage
0,0,what does ocd mean,One of the most helpful things in my own recov...
1,1,is it good to talk about an ocd,One of the most helpful things in my own recov...
2,2,does ocd really help recovery,One of the most helpful things in my own recov...
3,3,what is being a positive person,There is something powerful about knowing that...
4,4,who is an inspirational person,There is something powerful about knowing that...


## Negative Mining

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256

model


/home/ubuntu/gitrepos/semantic-search/ENV/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:00<00:00, 113kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.99k/3.99k [00:00<00:00, 3.85MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 100kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265M/265M [00:04<00:00, 61.0MB/s]
Downloading: 100%|███

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [8]:
import pinecone

API_KEY = "55e5d3b2-c5f7-494d-8323-3c79b45150a6"

pinecone.init(
    api_key=API_KEY,
    environment='us-west1-gcp'
)
# create a new genq index if does not already exist
if 'negative-mine' not in pinecone.list_indexes():
    pinecone.create_index(
        'negative-mine',
        dimension=model.get_sentence_embedding_dimension(),
        metric='dotproduct',
        pods=1  # increase for faster mining
    )
# connect
index = pinecone.Index('negative-mine')

In [ ]:
def get_text(df):
    pairs = []
    for index, row in df.iterrows():
        pairs.append((row['query'], row['passage']))

In [ ]:
pair_gen = get_text()  # generator that loads (query, passage) pairs

pairs = []
to_upsert = []
passage_batch = []
id_batch = []
batch_size = 64  # encode and upload size

for i, (query, passage) in enumerate(pairs_gen):
    pairs.append((query, passage))
    # we do this to avoid passage duplication in the vector DB
    if passage not in passage_batch: 
        passage_batch.append(passage)
        id_batch.append(str(i))
    # on reaching batch_size, we encode and upsert
    if len(passage_batch) == batch_size:
        embeds = model.encode(passage_batch).tolist()
        # upload to index
        index.upsert(vectors=list(zip(id_batch, embeds)))
        # refresh batches
        passage_batch = []
        id_batch = []
        
# check number of vectors in the index
index.describe_index_stats()